# Text Mining of BBC News Data

## Part 2: Bag-of-Words Text Vectorization


## The Analyzer Object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer

In [ ]:
test_sentence = "C'est l'été au Brésil!"

In [ ]:
word_analyzer = CountVectorizer().build_analyzer()
word_analyzer(test_sentence)

In [ ]:
word_analyzer = CountVectorizer(strip_accents="unicode", ngram_range=(2, 2)).build_analyzer()
word_analyzer(test_sentence)

In [ ]:
word_analyzer = CountVectorizer(strip_accents="unicode", ngram_range=(1, 2)).build_analyzer()
word_analyzer(test_sentence)

In [ ]:
word_analyzer = CountVectorizer(ngram_range=(2, 2)).build_analyzer()
word_analyzer(test_sentence)

## Analyzer = Preprocessor + Tokenizer (+ Token Filtering) (+ n-grams extraction)

In [ ]:
vectorizer = CountVectorizer(strip_accents="unicode", lowercase=True)

In [ ]:
test_sentence = "C'est l'été au Brésil!"

### Exercises

- Type `vectorizer.build_<TAB>` to see the list of methods of the vectorizer object;

- Use the vectorizer to build a preprocessor object and apply it to the test sentence: which transformations do you notice?

- Use the vectorizer to build a tokenizer object and apply it to the preprocessed test sentence from the previous step;

- Compare the results of the previous two steps with the output of the analyzer applied to the original test sentence.

In [ ]:
# %load notebook_solutions/build_preprocessor.py

In [ ]:
# %load notebook_solutions/build_tokenizer.py

## Vectorization of a Full Dataset

In [ ]:
from pathlib import Path


bbc_folder_path = Path("bbc")
text_filepaths = sorted(bbc_folder_path.glob("*/*.txt"))

Instead of decoding the text manually as we did before, we will path the filenames directly to the vectorizer and let it decode using the encoding of our choice and ignore decoding errors (as we did previously):

In [ ]:
%%time

vectorizer = CountVectorizer(encoding="utf-8", input="filename",
                             decode_error="ignore",
                             min_df=5, max_df=0.8)

vectorizer.fit(text_filepaths)

In [ ]:
type(vectorizer.vocabulary_)

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
vocabulary_items = sorted(vectorizer.vocabulary_.items())

In [ ]:
vocabulary_items[:10]

In [ ]:
vocabulary_items[5000:5010]

In [ ]:
vocabulary_items[-10:]

In [ ]:
vectorized_docs = vectorizer.transform(text_filepaths)

In [ ]:
vectorized_docs

In [ ]:
first_doc = vectorized_docs[0, :]

In [ ]:
first_doc

In [ ]:
first_doc.toarray()

**Question**: why does the vectorizer return a sparse matrix instead of a regular (dense) array?

In [ ]:
first_doc.toarray().shape

Let's have a look at the indices of the non-zero values in the vectorized first document:

In [ ]:
first_doc.indices

Each dimension corresponds to a word in the vocabulary. We can retrieve the "feature name" for a specific dimension number as follows:

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names[:10]

In [ ]:
feature_names[5360]

It's possible to inverse a full vectorized document at once:

In [ ]:
vectorizer.inverse_transform(first_doc)

**Exercise**

Print the text of the original first document. And compare to the "inversed transformed" document (can you find the same words?):

**Question**:

- Why do you thing the result of the text vectorization procedure is called "Bag-of-Words" representation?

- What as the main limitation of the "Bag-of-Words" representation?

- Can you come up with a pair of sentence with completely different meanings but that would share the same Bag of Words vector?

## TF-IDF Normalization

- TF-IDF stands for **"Term Frequency" - "Inverse Document Frequency"**.

- **Term Frequency** is the number of times a term or token appears in a given document;

- **Document Frequency** is the number of times a term or token appears across all the documents of the corpus.


**Note**: depending on the context, frequencies can either be:

- **absolute** values (**integer counts**) or
- **relative** values (**floating point numbers between 0 and 1** for **ratio** between two quantities).


TF-IDF is a normalization procedures that transforms integer term frequency counts into a reweights term frequencies such that very frequent words (such as "the", "a", "and", "it"... in English) have lower importance than rarer words.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


tfidf_transformer = TfidfTransformer()
tfidf_docs = tfidf_transformer.fit_transform(vectorized_docs)
tfidf_docs

In [ ]:
first_doc.toarray()

In [ ]:
first_tfidf_doc = tfidf_docs[0, :]
first_tfidf_doc.toarray()

In [ ]:
tfidf_transformer.idf_

**Exercises**:
    
Using the `vectorizer.get_feature_names()` vector and the `tfidf_transformer.idf_`, compute the list of the top 10 least and most informative words in the BBC corpus.

Hint: you can put the two vectors in a `pandas.DataFrame` and sort by IDF values.